In [17]:
import sys
import os

import ccxt
import backtesting as bt
from backtesting import Backtest, Strategy
import talib
import pandas_ta as ta

import pandas as pd
import numpy as np

In [18]:

def get_data(symbol, timeframe, limit):
    exchange = ccxt.kraken()
    exchange.load_markets()

    data = exchange.fetch_ohlcv('BTC/USDT', timeframe=timeframe, limit=limit)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']).rename(columns={
        'timestamp': 'timestamp',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [19]:
data = get_data('BTC/USDT', '1h', 1000)

# BBands

In [20]:

class BBANDS_strategy_v1(Strategy):
    ############################ Parameters ####################################
    BB_SMA = 20                         # Bollinger bands SMA
    BB_STD = 2                        # Bollinger bands standard deviation
    BB_MAX_BANDWIDTH = 5                # Bollinger bands maximum volatility allowed
    
    min_volatility = 0.3
    max_buy_perc = 0.2
    min_sell_perc = 0.25

    ############################ Utilities ####################################
    # For logging
    def log(self, txt, dt=None):
        dt = dt or self.data.index[-1]
        # print(f'{dt.strftime("%Y-%m-%d %H:%M:%S")}: {txt}')
    
    ############################ Strategy: Calculate the indicators ####################################

    def init(self):
        
        # print(f"BB_SMA: {self.BB_SMA}, BB_STD: {self.BB_STD}, BB_MAX_BANDWIDTH: {self.BB_MAX_BANDWIDTH}")
        # print(f"min_volatility: {self.min_volatility}, max_buy_perc: {self.max_buy_perc}, min_sell_perc: {self.min_sell_perc}")
        
        self.upper = self.I(lambda df, length, std: df.ta.bbands(close = df['Close'], length=length, std=std).iloc[:, 2],
                            self.data.df, self.BB_SMA, self.BB_STD, name='upper')
        
        self.lower = self.I(lambda df, length, std: df.ta.bbands(close = df['Close'], length=length, std=std).iloc[:, 0],
                            self.data.df, self.BB_SMA, self.BB_STD, name='lower')
        
        self.volatility = self.I(lambda df, length, std: df.ta.bbands(close = df['Close'], length=length, std=std).iloc[:, 1],
                                 self.data.df, self.BB_SMA, self.BB_STD, name='volatility', plot=True, overlay=False)

        self.high_limit = self.I(lambda upper, lower: upper + (upper - lower) / 2, self.upper, self.lower, name='high_limit')
        self.low_limit = self.I(lambda upper, lower: lower - (upper - lower) / 2, self.upper, self.lower, name='low_limit')
        self.close_percentage = self.I(lambda close, low_limit, high_limit: np.clip((close - low_limit) / (high_limit - low_limit), 0, 1), self.data.df['Close'], self.low_limit, self.high_limit, name='close_percentage')
        self.volatility_scaled = self.I(lambda volatility: np.clip(volatility / (100 / self.BB_MAX_BANDWIDTH), 0, 1), self.volatility, name='volatility_scaled')
        
        self.buy_signal = self.I(lambda volatility_scaled, close_percentage: (volatility_scaled > self.min_volatility) & (close_percentage < self.max_buy_perc), self.volatility_scaled, self.close_percentage, name='buy_signal')
        self.sell_signal = self.I(lambda close_percentage: (close_percentage > self.min_sell_perc), self.close_percentage, name='sell_signal')
    
    

    def next(self):
        
        self.log(f"Close: {self.data.Close[-1]}, position: {self.position.size}, cash: {self._broker.margin_available}")
        
        if self.position.size == 0:
            if self.buy_signal[-1]:
                self.log("BUY")
                self.buy()
                
        elif self.position.size > 0:
            if self.sell_signal[-1]:
                self.log("SELL")
                self.trades[0].close()

In [21]:
# BBANDS_strategy.BB_SMA = 40
# BBANDS_strategy.BB_STD = 2
# BBANDS_strategy.BB_MAX_BANDWIDTH = 9
# BBANDS_strategy.min_volatility = 0.6
# BBANDS_strategy.max_buy_perc = 0.3
# BBANDS_strategy.min_sell_perc = 0.2

bt = Backtest(
    data, 
    BBANDS_strategy_v1,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run()
bt.plot()

Row(id='9949', ...)

In [22]:
strat

Start                     2024-04-20 03:00:00
End                       2024-05-20 02:00:00
Duration                     29 days 23:00:00
Exposure Time [%]                    6.111111
Equity Final [$]                 98902.030775
Equity Peak [$]                      100000.0
Return [%]                          -1.097969
Buy & Hold Return [%]                4.221961
Return (Ann.) [%]                  -12.189752
Volatility (Ann.) [%]                2.480163
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -1.37474
Avg. Drawdown [%]                    -1.37474
Max. Drawdown Duration       25 days 12:00:00
Avg. Drawdown Duration       25 days 12:00:00
# Trades                                   12
Win Rate [%]                        41.666667
Best Trade [%]                       0.360375
Worst Trade [%]                     -0.678263
Avg. Trade [%]                    

## Optimization

In [24]:
import multiprocessing as mp
mp.set_start_method('fork')

RuntimeError: context has already been set

In [25]:
bt = Backtest(
    data, 
    BBANDS_strategy_v1,
    commission=0.00075, 
    cash=100000, 
    )

stats = bt.optimize(
                    BB_SMA=range(10, 70, 5),
                    BB_STD=range(1, 10, 2),
                    BB_MAX_BANDWIDTH=range(1, 10, 1),
                    min_volatility=list(np.arange(0, 0.5, 0.05)),
                    max_buy_perc=list(np.arange(0.1, 1, 0.05)),
                    min_sell_perc=list(np.arange(0, 0.5, 0.05)),
                    maximize='Return [%]',
                    max_tries=500,
                    random_state=1) 

# pd.DataFrame(stats)
# stats._strategy

/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 501 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/11 [00:00<?, ?it/s]

/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)),

In [26]:
stats._strategy

<Strategy BBANDS_strategy_v1(BB_SMA=10,BB_STD=1,BB_MAX_BANDWIDTH=1,min_volatility=0.35000000000000003,max_buy_perc=0.40000000000000013,min_sell_perc=0.45)>

In [27]:
bt = Backtest(
    data, 
    BBANDS_strategy_v1,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run( BB_SMA=10, BB_STD=1, BB_MAX_BANDWIDTH=1, min_volatility=0.35, max_buy_perc=0.4, min_sell_perc=0.45)
bt.plot()

Row(id='11675', ...)

In [28]:
strat

Start                     2024-04-20 03:00:00
End                       2024-05-20 02:00:00
Duration                     29 days 23:00:00
Exposure Time [%]                   49.583333
Equity Final [$]                101559.018025
Equity Peak [$]                 102159.618025
Return [%]                           1.559018
Buy & Hold Return [%]                4.221961
Return (Ann.) [%]                   19.978889
Volatility (Ann.) [%]               22.308244
Sharpe Ratio                         0.895583
Sortino Ratio                        1.401282
Calmar Ratio                         3.438202
Max. Drawdown [%]                   -5.810854
Avg. Drawdown [%]                   -1.019127
Max. Drawdown Duration        8 days 00:00:00
Avg. Drawdown Duration        1 days 23:00:00
# Trades                                   68
Win Rate [%]                        76.470588
Best Trade [%]                       2.705413
Worst Trade [%]                     -4.581494
Avg. Trade [%]                    